# Key and Ciphertext saving

Load packages 

In [7]:
%load_ext autoreload
%autoreload 2

Using CPU version HEAAN
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
from time import time
import numpy as np
import fase
from fase import HEAAN as he
from fase.core.common import HEAANContext

Using CPU version HEAAN


Set up FHE context.  
Secret key is automatically stored.

In [32]:
logq = 120
logp = 30
logn = 14 # small logn for quick test
n = 1*2**logn
slots = n

t0 = time()
hec = HEAANContext(logn, logp, logq, boot=False, key_path="SecretKey.txt", rot_l=[1,2])
print(f"Generating HEAAN context took {time() - t0} seonds")
print(f"logn  {logn},  logp {logp},  logq{logq}")

saving secret key done.
HEAAN CKKS setup is ready
Generating HEAAN context took 9.384517431259155 seonds
logn  14,  logp 30,  logq120


Generate a ctxt and save to a file

In [33]:
ctxt = hec.encrypt([1,2,3])

he.SerializationUtils.writeCiphertext(ctxt, "ctxt.dat")

hec.decrypt(ctxt)

array([ 9.99998476e-01,  2.00000937e+00,  2.99999915e+00, ...,
        3.20836457e-07, -1.80759889e-06, -1.00124951e-06])

Second scheme using the stored key

In [38]:
hec2 = HEAANContext(logn, logp, logq, boot=False, is_owner=False)

do_reduction=False

ring2 = he.Ring()
scheme2 = he.Scheme(ring2, True)

sk2 = he.SecretKey('SecretKey.txt')

HEAAN CKKS setup is ready
loading secret key done.


In [39]:
scheme2.decrypt(sk2, ctxt)

(0.999998, 0.000002)

## Client

In [7]:
#from fase import HEAAN
import fase.HEAAN as he
import numpy as np
import os 
import pickle
from fase.hnrf.hetree import HNRF
from fase.hnrf import heaan_nrf
from fase.hnrf.tree import NeuralTreeMaker
import torch

from time import time

PATH_SK = "./"
CAM_NAMES= {1: "e",
           2: "e",
           3: "a",
           4: "e",
           5: "e",
           6: "e",
           7: "e",
           8: "a",
           9: "a",
           10:"e",
           11:"e",
           12:"e",
           13:"a",
           14:"e"}
DEBUG = False

class HETreeFeaturizer:
    """Featurizer used by the client to encode and encrypt data.
       모든 Context 정보를 다 필요로 함. 
       이것만 따로 class를 만들고 CKKS context 보내기 좀 귀찮은데? 
    """
    def __init__(self, comparator: np.ndarray,
                 scheme, 
                 ckks_parms,
                 use_symmetric_key=False):
        self.comparator = comparator
        self.scheme = scheme
        #self.encoder = encoder
        self._parms = ckks_parms
        self.use_symmetric_key = use_symmetric_key

    def encrypt(self, x: np.ndarray):
        features = x[self.comparator]
        features[self.comparator == -1] = 0
        features = list(features)

        ctx = self._encrypt(features)
        return ctx

    def _encrypt(self, val, n=None, logp=None, logq=None):
        if n == None: n = self._parms.n
        if logp == None: logp = self._parms.logp
        if logq == None: logq = self._parms.logq

        ctxt = he.Ciphertext()#logp, logq, n)
        vv = np.zeros(n) # Need to initialize to zero or will cause "unbound"
        vv[:len(val)] = val
        self.scheme.encrypt(ctxt, he.Double(vv), n, logp, logq)
        del vv
        return ctxt

    def save(self, path:str):
        pickle.dump(self.comparator, open(path, "wb"))


class Param():
    def __init__(self, n=None, logn=None, logp=None, logq=None, logQboot=None):
        self.n = n
        self.logn = logn
        self.logp = logp
        self.logq = logq 
        self.logQboot = logQboot
        if self.logn == None:
            self.logn = int(np.log2(n))


def decrypt(scheme, secretKey, enc, parms):
    featurized = scheme.decrypt(secretKey, enc)
    arr = np.zeros(parms.n, dtype=np.complex128)
    featurized.__getarr__(arr)
    return arr.real

def encrypt(scheme, val, parms):
    ctxt = he.Ciphertext()#logp, logq, n)
    vv = np.zeros(parms.n) # Need to initialize to zero or will cause "unbound"
    vv[:len(val)] = val
    scheme.encrypt(ctxt, he.Double(vv), parms.n, parms.logp, parms.logq)
    del vv
    return ctxt

In [8]:
class HEAAN_Encryptor():
    def __init__(self, key_path, 
                debug=True, tar=False):

        logq = 540
        logp = 30
        logn = 14
        n = 1*2**logn
        is_serialized = True
        self.model_dir = "/home/hoseung/Work/Kinect_BBS_demo/models/"

        self.parms = Param(n=n, logp=logp, logq=logq)
        self.key_path = key_path
        if debug: print("[ENCRYPTOR] key path", key_path)

        self.ring = he.Ring()
        self.secretKey = he.SecretKey(PATH_SK)
        #self.secretKey = he.SecretKey(self.ring, PATH_SK)
        #self.scheme = he.Scheme(self.secretKey, self.ring, is_serialized, key_path)
        #self.scheme.addLeftRotKey(self.secretKey, 1)
        self.scheme = he.Scheme(self.ring, is_serialized, key_path)
        self.algo = he.SchemeAlgo(self.scheme)
        self.scheme.loadLeftRotKey(1)
        
        # q_text ={"root_path":key_path + 'serkey/',
        #             "keys_to_share":FN_KEYS}
        # e_key.set()

        self.set_featurizers()
        self.load_scalers()

        if debug: print("[Encryptor] HEAAN is ready")

    def set_featurizers(self):
        scheme = self.scheme
        parms = self.parms

        featurizers =[]
        for action in range(1,15):
            cam = CAM_NAMES[action]
            Nmodel = pickle.load(open(self.model_dir+f"Nmodel_{action}_{cam}.pickle", "rb"))
            h_rf = HNRF(Nmodel)
            featurizers.append((f"{action}_{cam}", HETreeFeaturizer(h_rf.return_comparator(), scheme, parms)))
                
        self.featurizers = dict(featurizers)

    def load_scalers(self):
        
        scalers =[]
        for action in range(1,15):
            cam = CAM_NAMES[action]
            sc = pickle.load(open(self.model_dir+f'scaler_{action}_{cam}.pickle', "rb"))
            scalers.append((f"{action}_{cam}", sc))
            
        self.scalers = dict(scalers)


    def test_encrypt(self):
        scheme = self.scheme
        parms = self.parms
        from glob import glob
        #from bbsQt.model import kinect_utils as ku 
        from bbsQt.model import rec_utils as ru
        from bbsQt.model import data_preprocessing as dpre

        # Load test dataset
        testdata_dir = "/home/hoseung/Dropbox/DeepInsight/2021ETRI/BBS_data/TestDataset/"
        action = 1
        cam = CAM_NAMES[action]
        fns = glob(testdata_dir + f"{cam}_*_{action}_2_skeleton.pickle")
        skarr= pickle.load(open(fns[0], "rb"))

        sub = ru.smoothed_frame_N(skarr, 
                        nframe=8, 
                        shift=1)

        new_sks = ru.ravel_rec(sub)[np.newaxis, :]
        skeleton = new_sks[0]

        # Standardize
        skeleton = dpre.shift_to_zero(skeleton)
        body = dpre.measure_lengths(skeleton)
        skeleton /= body['body']

        #action = int(sk['action'])
        #cam = sk['cam']

        sc = self.scalers[f"{action}_{cam}"]
        fn = f"ctx_{action:02d}_{cam}.dat"
        
        featurizer = self.featurizers[f"{action}_{cam}"]
        
        print("Featurizing skeleton...")

        #skeleton = sk['skeleton']

        if DEBUG:
            scaled = sc.transform(skeleton)
        else:
            rav_sub = skeleton#[0]
            print("rav_sub", rav_sub.min(), rav_sub.max())
            scaled = sc.transform(rav_sub.reshape(1,-1))
        
        print("scaled", scaled.shape)
        print(scaled.min(), scaled.max())
        
        # Still some values can surpass 1.0. 
        # I need a more strict rule for standardization.. 
        # The following is an ad-hoc measure.
        
        sc0 = scaled[0]
        sc_min = min((sc0.min(), 0)) # shift if min is negative 
        sc0 -= sc_min
        sc0 /= sc0.max()*1.02 # Just to give some padding area

        #sc0 = (sc0 - sc0.min()) / (2*(sc0.max() - sc0.min())) + 0.5*np.mean(sc0)
        #print("MIN", sc0.min(), "MAX", sc0.max())
        
        print("finally scaled", sc0.min(), sc0.max())
        ctx1 = featurizer.encrypt(sc0)
        print("Saving this values...")
        he.SerializationUtils.writeCiphertext(ctx1, fn)


    def start_encrypt_loop(self, q1, q_text, q_answer, e_sk, e_enc, e_ans, e_enc_ans, debug=True):
        """
        When skeleton is ready (e_sk), get the skeleton from q1, 
        encrypt, and store it as ctx_{i}.dat file. 
        """
        scheme = self.scheme
        parms = self.parms

        i=0 # 이거 필요 있음? 원본 스켈레톤은 pickle로 저장 될 테니까 암호화된 버전을 굳이 따로따로 저장할 이유가? 
        while True:
            e_sk.wait()  ## FLOW CONTROL
            print("[Encryptor] good to go") 
            sk = q1.get()  ## FLOW CONTROL
            "++++++++++++++++++++++++++ SKELETON POINTS ++++++++++++++++++++++"
            print(sk)
            
            e_sk.clear()  ## FLOW CONTROL: reset skeleton event
            
            if not 'skeleton' in sk.keys():
                raise LookupError("Can't find skeleton in queue")    
            if debug: print("[Encryptor] Got a skeleton, Encrypting...")
            if debug: print("[Encryptor] Length of the skeleton:", len(sk["skeleton"]))
            action = int(sk['action'])
            cam = sk['cam']

            sc = self.scalers[f"{action}_{cam}"]
            fn = f"ctx_{action:02d}_{cam}_{i}.dat"
           
            featurizer = self.featurizers[f"{action}_{cam}"]
            if debug: print("Featurizing skeleton...")
            t0 = time()            

            skeleton = sk['skeleton']

            if DEBUG:
                scaled = sc.transform(skeleton)
            else:
                rav_sub = skeleton[0]
                scaled = sc.transform(rav_sub.reshape(1,-1))
            
            sc0 = scaled[0]
            sc0 = (sc0 - sc0.min()) / (2*(sc0.max() - sc0.min())) + 0.5*np.mean(sc0)
            print("MIN", sc0.min(), "MAX", sc0.max())
            ctx1 = featurizer.encrypt(sc0)
            pickle.dump(sc0, open("scaled.pickle", "wb"))
            if debug: print(f"Featurizing done in {time() - t0:.2f}s")
            if debug: print(ctx1.n, ctx1.logp, ctx1.logq)
            if debug: print("[Encryptor] Ctxt encrypted")


            he.SerializationUtils.writeCiphertext(ctx1, fn)
            if debug: print("[Encryptor] Ctxt wrote")

            if DEBUG:
                self.setup_eval(server_path="./")
                dec=decrypt(self.scheme, self.secretKey, ctx1, self.parms)
                print("[encryptor] decrypt ctxt1", dec[:20])
                del ctx1
                ctx1 = he.Ciphertext(self.parms.logp, self.parms.logq, self.parms.n)
                he.SerializationUtils.readCiphertext(ctx1, fn)
                dec=decrypt(self.scheme, self.secretKey, ctx1, self.parms)
                print("[encryptor] decrypt ctxt1 again", dec[:20])
                #show_file_content(fn)
                #t0 =time()
                results = self.run_model(action, cam, ctx1)
                print(f"Prediction took {time()-t0:.2f} seconds")

                # Save prediction
                fn_preds = []
                for i, pred in enumerate(results):
                    print("PRED", i, pred)
                    #fn = self.server_path+f"pred_{i}.dat"
                    fn = f"pred_{i}.dat"
                    he.SerializationUtils.writeCiphertext(pred, fn)
                    fn_preds.append(fn)
                    print(pred)
                
                #fn_preds = [f"pred_{i}.dat" for i in range(5)]
                # Load predictions
                print("fn_preds", fn_preds)
                logq = 180
                preds=[]
                for fn_ctx in fn_preds:
                    print("[encryptor] make an empty ctxt")
                    ctx_pred = he.Ciphertext(ctx1.logp, logq, ctx1.n) # 나중에 오는 애는 logq가 다를 수도 있음
                    print("[encryptor] load ctxt", fn_ctx)
                    he.SerializationUtils.readCiphertext(ctx_pred, fn_ctx)
                    print("[encryptor] decrypt ctxt", ctx_pred)
                    dec=decrypt(self.scheme, self.secretKey, ctx_pred, self.parms)
                    print("[encryptor] append decrypted ctxt")
                    preds.append(np.sum(dec))# Must sum the whole vector. partial sum gives wrong answer
                    print("[encryptor] decrypted prediction array", dec[:10])
                    del ctx_pred
                del ctx1 
            
                print(f"Predicted score: {np.argmax(preds)}")

            q1.put({"fn_enc_skeleton": fn})  ## FLOW CONTROL
            if debug: print("[Encryptor] skeleton encrypted and saved as", fn)
            e_enc.set()  ## FLOW CONTROL: encryption is done and file is ready
            
            if debug: print("[Encryptor] Waiting for prediction...")

            # Decrypt answer
            e_enc_ans.wait()  ## FLOW CONTROL
            preds = []
            fn_preds = q_text.get()  ## FLOW CONTROL

            #fn_preds = [f"pred_{i}.dat" for i in range(5)]
            # Load predictions
            print("fn_preds", fn_preds)
            logq = 180
            preds=[]
            for fn_ctx in fn_preds:
                print("[encryptor] make an empty ctxt")
                ctx_pred = he.Ciphertext(ctx1.logp, logq, ctx1.n) # 나중에 오는 애는 logq가 다를 수도 있음
                print("[encryptor] load ctxt", fn_ctx)
                he.SerializationUtils.readCiphertext(ctx_pred, fn_ctx)
                print("[encryptor] decrypt ctxt", ctx_pred)
                dec=decrypt(self.scheme, self.secretKey, ctx_pred, self.parms)
                print("[encryptor] append decrypted ctxt")
                preds.append(np.sum(dec))# Must sum the whole vector. partial sum gives wrong answer
                print("[encryptor] decrypted prediction array", dec[:10])
                del ctx_pred
            del ctx1 

            print("preds", preds)
            ans_str = f"Predicted score: {np.argmax(preds)}"
            print(ans_str)
            e_enc_ans.clear()  ## FLOW CONTROL
            
            # 복호화된 결과 QT로 전송
            q_answer.put(ans_str)  ## FLOW CONTROL
            e_ans.set()  ## FLOW CONTROL

            i+=1
    
    def setup_eval(self, server_path="./"):
        logq = 540
        logp = 30
        logn = 14
        n = 1*2**logn

        self.parms2 = Param(n=n, logp=logp, logq=logq)
        self.server_path2 = server_path
        self.key_path2 = server_path + 'serkey/'
        print("[ENCRYPTOR] key path", self.key_path2)

        self.ring2 = he.Ring()
        
        self.scheme2 = he.Scheme(self.ring2, True, self.server_path2)
        self.algo2 = he.SchemeAlgo(self.scheme2)
        self.scheme2.loadLeftRotKey(1)

    def load_models(self):
        self.models = {}
        dilatation_factor = 10
        polynomial_degree = 10

        self.my_tm_tanh = NeuralTreeMaker(torch.tanh, 
                            use_polynomial=True,
                            dilatation_factor=dilatation_factor, 
                            polynomial_degree=polynomial_degree)
        
    def load_model(self, action, cam):
        
        print("[Evaluator] Loading trained NRF models")

        t0 = time()
        fn = f"./models/Nmodel_{action}_{cam}.pickle"
        Nmodel = pickle.load(open(fn, "rb"))
        #print("Loaded a model...", fn)
        
        h_rf = HNRF(Nmodel)
        #print("[EVAL.model_loader] HRF loaded for class", action)
        nrf_evaluator = heaan_nrf.HETreeEvaluator.from_model(h_rf,
                                                            self.scheme2,
                                                            self.parms2,
                                                            self.my_tm_tanh.coeffs,
                                                            do_reduction = False,
                                                            #save_check=True
                                                            )
        print("[EVAL.model_loader] HNRF model loaded for class", action)
            
        #allmodels.append((f"{action}",nrf_evaluator))
        self.models.update({f"{action}_{cam}":nrf_evaluator})    
        
        print("updated models", self.models)    


    def run_model(self, action, cam, ctx):
        self.load_models()
        print("Running model for class", action)
        try:
            model = self.models[f"{action}_{cam}"]
        except:
            self.load_model(action, cam)
            print(f"Loading model for class {action} and camera {cam}")
            model = self.models[f"{action}_{cam}"]

        #featurizer = self.models[f"{cc}"]['featurizer']
        print("[Evaluator] running model...")
        #ctx = featurizer.encrypt(data)
        return model(ctx)
        #return self.predict(data)


In [9]:
encryptor = HEAAN_Encryptor("./")

[ENCRYPTOR] key path ./
loading secret key done.
[Encryptor] HEAAN is ready


In [10]:
encryptor.test_encrypt()

Featurizing skeleton...
rav_sub -1.7155419709998272 1.588739030602214
scaled (1, 240)
0.08759097299201701 1.15995010857075
finally scaled 0.07403206594735384 0.9803921568627451
VV
[0.67849136 0.68550511 0.66578847 ... 0.         0.         0.        ]
16384 30 540
Saving this values...


In [35]:
model = encryptor.models[1]

AttributeError: 'HEAAN_Encryptor' object has no attribute 'models'

In [6]:
from time import time
import numpy as np
import fase
from fase import HEAAN as he
from fase.core.common import HEAANContext

logq = 540
logp = 30
logn = 14
n = 1*2**logn
is_serialized = True


# Bare heaan

In [32]:
if not os.path.isdir("./serkey/"): os.mkdir("./serkey/")

In [29]:

import os


In [12]:

ring = he.Ring()

secretKey = he.SecretKey(ring, "./serkey/sk.dat")
#self.secretKey = he.SecretKey(self.ring, PATH_SK)
scheme = he.Scheme(secretKey, ring, is_serialized, './serkey/')
scheme.addLeftRotKey(secretKey, 1)

saving secret key done.


In [9]:

ring = he.Ring()
scheme = he.Scheme(ring, True, './')
secretKey = he.SecretKey("./")

loading secret key done.


In [14]:
val = [0.67849136, 0.68550511, 0.66578847]

ctxt = he.Ciphertext()#logp, logq, n)
vv = np.zeros(n) # Need to initialize to zero or will cause "unbound"
vv[:len(val)] = val
print("VV")
print(vv)


VV
[0.67849136 0.68550511 0.66578847 ... 0.         0.         0.        ]


In [15]:

scheme.encrypt(ctxt, he.Double(vv), n, logp, logq)

In [23]:
he.SerializationUtils.writeCiphertext(ctxt, "test.dat")

In [6]:
scheme.loadLeftRotKey(1)

## Server

### scheme without secretkey

In [2]:
ring2 = he.Ring()
       
scheme2 = he.Scheme(ring2, True, "./")
algo2 = he.SchemeAlgo(scheme2)
scheme2.loadLeftRotKey(1)

# Do some computation including rotation.
secretKey = he.SecretKey("./sk.dat")

loading secret key done.


In [25]:
ctxt2 = he.Ciphertext(logp, logq, n)
he.SerializationUtils.readCiphertext(ctxt2, "test.dat")

In [16]:
scheme2.decrypt(secretKey, ctxt)

(0.678486, -0.000002)

In [14]:
scheme2.decrypt(secretKey, ctxt)

(0.678489, -0.000001)

In [27]:
scheme2.leftRotateFastAndEqual(ctxt, 1)

In [28]:
scheme2.decrypt(secretKey, ctxt)

(0.685504, -0.000002)

## MultPlain test

### multByConst

In [17]:
val = [1,2,3,4,5,6,7]

ctxt = he.Ciphertext()#logp, logq, n)
vv = np.zeros(n) # Need to initialize to zero or will cause "unbound"
vv[:len(val)] = val
print(vv)

scheme.encrypt(ctxt, he.Double(vv), n, logp, logq)



[1. 2. 3. ... 0. 0. 0.]


In [34]:
scheme.multByConstVecAndEqual?

Docstring: multByConstVecAndEqual(self: fase.HEAAN.Scheme, arg0: Ciphertext, arg1: fase.HEAAN.ComplexDouble, arg2: int) -> None
Type:      method


In [18]:
val2 = [10,9,8,7,6,5,4]
v2 = np.zeros(n) # Need to initialize to zero or will cause "unbound"
v2[:len(val2)] = val2
print(v2)

scheme.multByConstVecAndEqual(ctxt, he.Double(v2), logp)

[10.  9.  8. ...  0.  0.  0.]


(10.000018, 0.000011)

In [24]:
ddd = scheme.decrypt(secretKey, ctxt)
arr = np.zeros(n, dtype=np.complex128)
ddd.__getarr__(arr)
res = arr.real

In [25]:
res

array([ 1.00000177e+01,  1.80000062e+01,  2.39999894e+01, ...,
        2.70124330e-16, -4.24238630e-13,  3.34435018e-14])

In [23]:
ddd.print(10)

[(10,1.07766e-05), (18,-2.05715e-05), (24,2.17227e-05), (28,-1.73329e-05), (30,-7.05997e-06), (30,-1.77465e-05), (28,1.50483e-06), (-2.90546e-13,-2.07096e-13), (9.3802e-14,2.66266e-13), (-1.72765e-13,8.31416e-14)]


In [45]:
dd = he.Double(v2)

In [47]:
scheme.encrypt(ctxt, he.Double(v2), n, logp, logq)

In [51]:
scheme.encrypt(ctxt, he.Double(v2), n, logp, [1,2,3])

TypeError: encrypt(): incompatible function arguments. The following argument types are supported:
    1. (self: fase.HEAAN.Scheme, arg0: Ciphertext, arg1: fase.HEAAN.ComplexDouble, arg2: int, arg3: int, arg4: int) -> None
    2. (self: fase.HEAAN.Scheme, arg0: Ciphertext, arg1: fase.HEAAN.Double, arg2: int, arg3: int, arg4: int) -> None

Invoked with: <fase.HEAAN.Scheme object at 0x7fe3ebb2f7b0>, <class.Ciphertext logp: 30 logq: 540 n: 16384>, 10.000000, 16384, 30, [1, 2, 3]

In [53]:
scheme.multByConstVecAndEqual(ctxt, he.ComplexDouble(v2), logp)

In [54]:
scheme.decrypt(secretKey, ctxt)

(99.999995, 0.000031)